## stanford sentiment model

In [1]:
import json
import numpy as np
import pandas as pd
from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP('../stanford-corenlp-full-2018-10-05')

def SSTscore(s):
    res = nlp.annotate(s,
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json'
                   })
    d = json.loads(res)
    return float(d['sentences'][0]['sentimentValue'])

In [51]:
testtext=[]
testlabel=[]
tmp=[]
with open('../stanford-corenlp-full-2018-10-05/stanfordSentimentTreebank/mytest.tsv','r') as f:
    for line in f:
        s = line.split('\t')
        testtext.append(s[0])
        testlabel.append(float(s[1].strip('\n')))

In [62]:
SSTscore('best')

4.0

In [22]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from itertools import chain
from torchtext import data


class RNN_VAE(nn.Module):
    """
    1. Hu, Zhiting, et al. "Toward controlled generation of text." ICML. 2017.
    2. Bowman, Samuel R., et al. "Generating sentences from a continuous space." arXiv preprint arXiv:1511.06349 (2015).
    3. Kim, Yoon. "Convolutional neural networks for sentence classification." arXiv preprint arXiv:1408.5882 (2014).
    """

    def __init__(self, n_vocab, h_dim, z_dim, p_word_dropout=0.3, unk_idx=0, pad_idx=1, start_idx=2, eos_idx=3, max_sent_len=15, pretrained_embeddings=None, freeze_embeddings=False, gpu=False):
        super(RNN_VAE, self).__init__()

        self.UNK_IDX = unk_idx
        self.PAD_IDX = pad_idx
        self.START_IDX = start_idx
        self.EOS_IDX = eos_idx
        self.MAX_SENT_LEN = max_sent_len

        self.n_vocab = n_vocab
        self.h_dim = h_dim
        self.z_dim = z_dim
        self.p_word_dropout = p_word_dropout

        self.gpu = gpu

        """
        Word embeddings layer
        """
        if pretrained_embeddings is None:
            self.emb_dim = h_dim
            self.word_emb = nn.Embedding(n_vocab, h_dim, self.PAD_IDX)
        else:
            self.emb_dim = pretrained_embeddings.size(1)
            self.word_emb = nn.Embedding(n_vocab, self.emb_dim, self.PAD_IDX)

            # Set pretrained embeddings
            self.word_emb.weight.data.copy_(pretrained_embeddings)

            if freeze_embeddings:
                self.word_emb.weight.requires_grad = False

        """
        Encoder is GRU with FC layers connected to last hidden unit
        """
        self.encoder = nn.GRU(self.emb_dim, h_dim)
        self.q_mu = nn.Linear(h_dim, z_dim)
        self.q_logvar = nn.Linear(h_dim, z_dim)

        """
        Decoder is GRU with `z` appended at its inputs
        """
        self.decoder = nn.GRU(self.emb_dim+z_dim, z_dim, dropout=0.3)
        self.decoder_fc = nn.Linear(z_dim, n_vocab)


        """
        Grouping the model's parameters: separating encoder, decoder
        """
        self.encoder_params = chain(
            self.encoder.parameters(), self.q_mu.parameters(),
            self.q_logvar.parameters()
        )

        self.decoder_params = chain(
            self.decoder.parameters(), self.decoder_fc.parameters()
        )

        self.vae_params = chain(
            self.word_emb.parameters(), self.encoder_params, self.decoder_params
        )
        self.vae_params = filter(lambda p: p.requires_grad, self.vae_params)

        """
        Use GPU if set
        """
        if self.gpu:
            self.cuda()

    def forward_encoder(self, inputs):
        """
        Inputs is batch of sentences: seq_len x mbsize
        """    
        inputs = self.word_emb(inputs)
        return self.forward_encoder_embed(inputs)

    def forward_encoder_embed(self, inputs):
        """
        Inputs is embeddings of: seq_len x mbsize x emb_dim
        """
        _, h = self.encoder(inputs, None)

        # Forward to latent
        h = h.view(-1, self.h_dim)
        mu = self.q_mu(h)
        logvar = self.q_logvar(h)

        return mu, logvar

    def sample_z(self, mu, logvar):
        """
        Reparameterization trick: z = mu + std*eps; eps ~ N(0, I)
        """
        eps = Variable(torch.randn(self.z_dim))
        eps = eps.cuda() if self.gpu else eps
        z = mu + torch.exp(logvar/2) * eps
        return z/z.pow(2).sum().pow(0.5)

    def sample_z_prior(self, mbsize):
        """
        Sample z ~ p(z) = N(0, I)
        """
        z = Variable(torch.randn(mbsize, self.z_dim))
        z = z.cuda() if self.gpu else z
        return z/z.pow(2).sum().pow(0.5)

    def forward_decoder(self, inputs, z):
        """
        Inputs must be embeddings: seq_len x mbsize
        """
        dec_inputs = self.word_dropout(inputs)

        # Forward
        seq_len = dec_inputs.size(0)

        # 1 x mbsize x z_dim
        init_h = z.unsqueeze(0)
        inputs_emb = self.word_emb(dec_inputs)  # seq_len x mbsize x emb_dim
        inputs_emb = torch.cat([inputs_emb, init_h.repeat(seq_len, 1, 1)], 2)

        outputs, _ = self.decoder(inputs_emb, init_h)
        seq_len, mbsize, _ = outputs.size()

        outputs = outputs.view(seq_len*mbsize, -1)
        y = self.decoder_fc(outputs)
        y = y.view(seq_len, mbsize, self.n_vocab)

        return y
    '''
    def recon_latent(self, inputs, z):
        """
        Inputs must be embeddings: seq_len x mbsize
        """
        dec_inputs = self.word_dropout(inputs)

        # Forward
        seq_len = dec_inputs.size(0)

        # 1 x mbsize x z_dim
        init_h = z.unsqueeze(0)
        inputs_emb = self.word_emb(dec_inputs)  # seq_len x mbsize x emb_dim
        inputs_emb = torch.cat([inputs_emb, init_h.repeat(seq_len, 1, 1)], 2)

        outputs, _ = self.decoder(inputs_emb, init_h)
        #seq_len, mbsize, _ = outputs.size()

        #outputs = outputs.view(seq_len*mbsize, -1)
        #y = self.decoder_fc(outputs)
        #y = y.view(seq_len, mbsize, self.n_vocab)

        return outputs
    '''
    
    def forward_VAE(self, sentence):
        """
        Params:
        -------
        sentence: sequence of word indices.
        use_c_prior: whether to sample `c` from prior or from `discriminator`.

        Returns:
        --------
        recon_loss: reconstruction loss of VAE.
        kl_loss: KL-div loss of VAE.
        """
        self.train()

        mbsize = sentence.size(1)

        # sentence: '<start> I want to fly <eos>'
        # enc_inputs: '<start> I want to fly <eos>'
        # dec_inputs: '<start> I want to fly <eos>'
        # dec_targets: 'I want to fly <eos> <pad>'
        pad_words = Variable(torch.LongTensor([self.PAD_IDX])).repeat(1, mbsize)
        pad_words = pad_words.cuda() if self.gpu else pad_words

        enc_inputs = sentence
        dec_inputs = sentence
        dec_targets = torch.cat([sentence[1:], pad_words], dim=0)

        # Encoder: sentence -> z
        mu, logvar = self.forward_encoder(enc_inputs)
        z = self.sample_z(mu, logvar)

        # Decoder: sentence -> y
        y = self.forward_decoder(dec_inputs, z)       
        
        recon_loss = F.cross_entropy(
            y.view(-1, self.n_vocab), dec_targets.view(-1), size_average=True
        )
        kl_loss = torch.mean(0.5 * torch.sum(torch.exp(logvar) + mu**2 - 1 - logvar, 1))

        return recon_loss, kl_loss
    
    def forward_RL(self,t):
        z = self.sample_z_prior(1)
        sample_idxs, ally = self.sample_sentence_RL(z,temp=t)
        sample_sent = ' '.join([TEXT.vocab.itos[i] for i in sample_idxs])
        sample_idxs = torch.tensor(sample_idxs,device=inputs.device)

        new_recon_loss = F.cross_entropy(ally, sample_idxs, size_average=True)

        return new_recon_loss,sample_sent
    
    def forward_RL_batch(self,t,batch_size):
        loss = torch.zeros(batch_size)
        sents = []
        for i in range(batch_size):
            z = self.sample_z_prior(1)
            sample_idxs, ally = self.sample_sentence_RL(z,temp=t)
            sample_sent = ' '.join([TEXT.vocab.itos[i] for i in sample_idxs])
            sample_idxs = torch.tensor(sample_idxs,device=inputs.device)
            new_recon_loss = F.cross_entropy(ally, sample_idxs, size_average=True)
            loss[i] = new_recon_loss
            sents.append(sample_sent)
        return loss, sents


    def generate_sentences(self, batch_size):
        """
        Generate sentences and corresponding z of (batch_size x max_sent_len)
        """
        samples = []

        for _ in range(batch_size):
            z = self.sample_z_prior(1)
            samples.append(self.sample_sentence(z, raw=True))

        return samples

    def sample_sentence_RL(self, z, raw=False, temp=1):
        """
        Sample single sentence from p(x|z,c) according to given temperature.
        `raw = True` means this returns sentence as in dataset which is useful
        to train discriminator. `False` means that this will return list of
        `word_idx` which is useful for evaluation.
        """
        #self.eval()
        self.train()

        word = torch.LongTensor([self.START_IDX])
        word = word.cuda() if self.gpu else word
        word = Variable(word)  # '<start>'

        z = z.view(1, 1, -1)
        h=z
        
        if not isinstance(h, Variable):
            h = Variable(h)

        outputs = []
        ally = []
        
        if raw:
            outputs.append(self.START_IDX)

        for i in range(self.MAX_SENT_LEN):
            emb = self.word_emb(word).view(1, 1, -1)
            emb = torch.cat([emb, z], 2)

            output, h = self.decoder(emb, h)
            y = self.decoder_fc(output).view(-1)
            y = F.softmax(y/temp, dim=0)

            idx = torch.multinomial(y,1)

            word = Variable(torch.LongTensor([int(idx)]))
            word = word.cuda() if self.gpu else word

            idx = int(idx)

            if not raw and idx == self.EOS_IDX:
                break

            outputs.append(idx)
            ally.append(y)

        # Back to default state: train
        #self.train()
        
        ally = torch.stack(ally)
        if raw:
            outputs = Variable(torch.LongTensor(outputs)).unsqueeze(0)
            return outputs.cuda() if self.gpu else outputs
        else:
            return outputs,ally
        
    def sample_sentence_VAE(self, z, raw=False, temp=1):
        """
        Sample single sentence from p(x|z,c) according to given temperature.
        `raw = True` means this returns sentence as in dataset which is useful
        to train discriminator. `False` means that this will return list of
        `word_idx` which is useful for evaluation.
        """
        self.eval()

        word = torch.LongTensor([self.START_IDX])
        word = word.cuda() if self.gpu else word
        word = Variable(word)  # '<start>'

        z= z.view(1, 1, -1)

        h = z

        if not isinstance(h, Variable):
            h = Variable(h)

        outputs = []

        if raw:
            outputs.append(self.START_IDX)

        for i in range(self.MAX_SENT_LEN):
            emb = self.word_emb(word).view(1, 1, -1)
            emb = torch.cat([emb, z], 2)

            output, h = self.decoder(emb, h)
            y = self.decoder_fc(output).view(-1)
            y = F.softmax(y/temp, dim=0)

            idx = torch.multinomial(y,1)

            word = Variable(torch.LongTensor([int(idx)]))
            word = word.cuda() if self.gpu else word

            idx = int(idx)

            if not raw and idx == self.EOS_IDX:
                break

            outputs.append(idx)

        # Back to default state: train
        self.train()

        if raw:
            outputs = Variable(torch.LongTensor(outputs)).unsqueeze(0)
            return outputs.cuda() if self.gpu else outputs
        else:
            return outputs

    def generate_soft_embed(self, mbsize, temp=1):
        """
        Generate soft embeddings of (mbsize x emb_dim) along with target z
        and c for each row (mbsize x {z_dim, c_dim})
        """
        samples = []
        targets_z = []

        for _ in range(mbsize):
            z = self.sample_z_prior(1)

            samples.append(self.sample_soft_embed(z, temp=1))
            targets_z.append(z)

        X_gen = torch.cat(samples, dim=0)
        targets_z = torch.cat(targets_z, dim=0)

        return X_gen, targets_z

    def sample_soft_embed(self, z, temp=1):
        """
        Sample single soft embedded sentence from p(x|z,c) and temperature.
        Soft embeddings are calculated as weighted average of word_emb
        according to p(x|z,c).
        """
        self.eval()

        z = z.view(1, 1, -1)

        word = torch.LongTensor([self.START_IDX])
        word = word.cuda() if self.gpu else word
        word = Variable(word)  # '<start>'
        emb = self.word_emb(word).view(1, 1, -1)
        emb = torch.cat([emb, z], 2)

        h = z

        if not isinstance(h, Variable):
            h = Variable(h)

        outputs = [self.word_emb(word).view(1, -1)]

        for i in range(self.MAX_SENT_LEN):
            output, h = self.decoder(emb, h)
            o = self.decoder_fc(output).view(-1)

            # Sample softmax with temperature
            y = F.softmax(o / temp, dim=0)

            # Take expectation of embedding given output prob -> soft embedding
            # <y, w> = 1 x n_vocab * n_vocab x emb_dim
            emb = y.unsqueeze(0) @ self.word_emb.weight
            emb = emb.view(1, 1, -1)

            # Save resulting soft embedding
            outputs.append(emb.view(1, -1))

            # Append with z and c for the next input
            emb = torch.cat([emb, z], 2)

        # 1 x 16 x emb_dim
        outputs = torch.cat(outputs, dim=0).unsqueeze(0)

        # Back to default state: train
        self.train()

        return outputs.cuda() if self.gpu else outputs

    def word_dropout(self, inputs):
        """
        Do word dropout: with prob `p_word_dropout`, set the word to '<unk>'.
        """
        if isinstance(inputs, Variable):
            data = inputs.data.clone()
        else:
            data = inputs.clone()

        # Sample masks: elems with val 1 will be set to <unk>
        mask = torch.from_numpy(
            np.random.binomial(1, p=self.p_word_dropout, size=tuple(data.size()))
                     .astype('uint8')
        )

        if self.gpu:
            mask = mask.cuda()

        # Set to <unk>
        data[mask] = self.UNK_IDX

        return Variable(data)

In [ ]:
fTEXT = data.Field(tokenize='spacy')
fLABEL = data.LabelField(tensor_type=torch.FloatTensor)
print("loading dataset mytrain.tsv for fasttext300...")
ftrain  = data.TabularDataset.splits(
        path='../stanford-corenlp-full-2018-10-05/stanfordSentimentTreebank/', 
        train='mytrain.tsv',
        format='tsv',
        fields=[('Text', fTEXT),('Label', fLABEL)])[0]
fTEXT.build_vocab(ftrain, max_size=60000, vectors="fasttext.en.300d",min_freq=1)
fLABEL.build_vocab(ftrain)
for a,b in fLABEL.vocab.stoi.items():
    fLABEL.vocab.stoi[a]=float(a)

    
mTEXT = data.Field(tokenize='spacy')
mLABEL = data.LabelField(tensor_type=torch.FloatTensor)
print("loading dataset mytrain.tsv for glove300...")
mtrain  = data.TabularDataset.splits(
        path='../stanford-corenlp-full-2018-10-05/stanfordSentimentTreebank/', 
        train='mytrain.tsv',
        format='tsv',
        fields=[('Text', mTEXT),('Label', mLABEL)])[0]
mTEXT.build_vocab(mtrain, max_size=60000, vectors="glove.6B.300d",min_freq=1)
mLABEL.build_vocab(mtrain)
for a,b in mLABEL.vocab.stoi.items():
    mLABEL.vocab.stoi[a]=float(a)

In [24]:
TEXT = mTEXT
LABEL = mLABEL

In [25]:
import math
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import argparse
from torchtext import data
SEED = 1

#torch.manual_seed(SEED)
#torch.cuda.manual_seed(SEED)

mb_size = 32
h_dim = 128
lr = 1e-4
lr_decay_every = 1000000
n_iter = 20000
log_interval = 1000
z_dim = 128


VAEmodel = RNN_VAE(
    len(mTEXT.vocab), h_dim, z_dim, p_word_dropout=0.3,max_sent_len=25,
    pretrained_embeddings=mTEXT.vocab.vectors, freeze_embeddings=False,
    gpu=True
)

VAEmodel.load_state_dict(torch.load('SSTmodel/allglove300_allfasttext300_VAE.bin'))

# Annealing for KL term
kld_start_inc = 3000
kld_weight = 0.01
kld_max = 0.15
kld_inc = (kld_max - kld_weight) / (n_iter - kld_start_inc)

trainer = optim.Adam(VAEmodel.vae_params, lr=lr)

train_iter = data.BucketIterator(
dataset=mtrain, batch_size=mb_size,
sort_key=lambda x: data.interleave_keys(len(x.src), len(x.trg)))

/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [27]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()      
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)      
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        y = self.fc(hidden.squeeze(0))
        return y
    
fINPUT_DIM = len(fTEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.4




fmodel = RNN(fINPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)
print("fmodel parameters: ")
print(fmodel.parameters)
pretrained_embeddings = fTEXT.vocab.vectors
fmodel.embedding.weight.data.copy_(pretrained_embeddings)
import torch.optim as optim
foptimizer = optim.Adam(fmodel.parameters(),lr=0.0003)
criterion = nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fmodel = fmodel.to(device)
criterion = criterion.to(device)


mINPUT_DIM = len(mTEXT.vocab)
mmodel = RNN(mINPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)
print("Shoesmodel parameters: ")
print(mmodel.parameters)
pretrained_embeddings = mTEXT.vocab.vectors
mmodel.embedding.weight.data.copy_(pretrained_embeddings)
import torch.optim as optim
moptimizer = optim.Adam(mmodel.parameters(),lr=0.0003)
criterion = nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mmodel = mmodel.to(device)
criterion = criterion.to(device)


#frnn = torch.load('frnn8', map_location=lambda storage, loc: storage) #force to load on CPU
#mrnn = torch.load('mrnn8', map_location=lambda storage, loc: storage) #force to load on CPU
print("loading fasttext300 model and glove300 model....")
fmodel.load_state_dict(torch.load('SSTmodel/SSTtrainall_fasttext300.bin')) #fasttext300
mmodel.load_state_dict(torch.load('SSTmodel/SSTtrainall.bin')) #glove300

fmodel = fmodel.to(device)
mmodel = mmodel.to(device)
criterion = criterion.to(device)


import spacy
nlp = spacy.load('en')

def predict_sentiment(sentence,model,TEXT):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    model.eval()
    prediction = model(tensor)
    return prediction.item()


def predict_word(word):
    print('frnn: ',predict_sentiment(word,frnn,fTEXT))
    print('mrnn:  ',predict_sentiment(word,mrnn,mTEXT))
    
    
###########################################################################

fmodel parameters: 
<bound method Module.parameters of RNN(
  (embedding): Embedding(20823, 300)
  (rnn): LSTM(300, 300, num_layers=2, dropout=0.4, bidirectional=True)
  (fc): Linear(in_features=600, out_features=1, bias=True)
  (dropout): Dropout(p=0.4)
)>
Shoesmodel parameters: 
<bound method Module.parameters of RNN(
  (embedding): Embedding(20823, 300)
  (rnn): LSTM(300, 300, num_layers=2, dropout=0.4, bidirectional=True)
  (fc): Linear(in_features=600, out_features=1, bias=True)
  (dropout): Dropout(p=0.4)
)>
loading fasttext300 model and glove300 model....


In [28]:
def save_model():
    if not os.path.exists('models/'):
        os.makedirs('models/')
    torch.save(model.state_dict(), 'models/{}.bin'.format('SST/allglove300_allglove200_RLVAE'))   

    
pre_weight = 1

batch = next(iter(train_iter))
inputs = batch.Text
labels = batch.Label

t=1

In [29]:
def train_RL(temperature,iteration):
    while 1:
        try:
            new_recon_loss,sample_sent = model.forward_RL(temperature)
        except:
            new_recon_loss,sample_sent = model.forward_RL(temperature)

        if temperature>1:
            temperature=temperature-0.0001

        if sample_sent:
            m0 = predict_sentiment(sample_sent,mmodel,mTEXT)
            f0 = predict_sentiment(sample_sent,fmodel,fTEXT)
            pre_dif_sample = abs(f0-m0)
        else:
            pre_dif_sample = 1/pre_weight

        if pre_dif_sample>1:
            #print("RL training sample sent: ",sample_sent)
            #print("RL training pre_dif: ",pre_dif_sample)
            #print('\n')
            loss = new_recon_loss*pre_dif_sample
            loss.backward()
            grad_norm = torch.nn.utils.clip_grad_norm(model.vae_params, 5)
            trainer.step()
            trainer.zero_grad()
            break

    if iteration%10==0:
        print(iteration)
        _,sample_sent = model.forward_RL(temperature)
        print('Sample: "{}"'.format(sample_sent))
        if sample_sent:
            m0 = predict_sentiment(sample_sent,mmodel,mTEXT)
            f0 = predict_sentiment(sample_sent,fmodel,fTEXT)
            pre_dif_sample = abs(f0-m0)
            print("RL mmodel sample prediction: ",m0)
            print("RL fmodel sample prediction: ",f0)
            print("RL sample abs dif: ",pre_dif_sample)
            print("\n")

In [30]:
new_recon_loss,sample_sents = VAEmodel.forward_RL_batch(1,32)


sents_dif=torch.zeros(32)
for i,s in enumerate(sample_sents):
    if s:
        m0 = predict_sentiment(s,mmodel,mTEXT)
        f0 = predict_sentiment(s,fmodel,fTEXT)
        pre_dif_sample = abs(f0-m0)
        if pre_dif_sample>1:
            sents_dif[i] = 0.1*min(pre_dif_sample,2)
    else:
        sents_dif[i] = 0

In [ ]:
sents_dif

In [ ]:
new_recon_loss

In [ ]:
torch.mean(torch.mul(sents_dif,new_recon_loss))

In [31]:
def train_RL_batch(temperature,iteration,batch_size,model):
    try:
        new_recon_loss,sample_sents = model.forward_RL_batch(temperature,batch_size)
    except:
        new_recon_loss,sample_sents = model.forward_RL_batch(temperature,batch_size)

    if temperature>1:
        temperature=temperature-0.0001

    sents_dif=torch.zeros(batch_size)
    for i,s in enumerate(sample_sents):
        if s:
            s = s.split('<pad>')[0]
            m0 = predict_sentiment(s,mmodel,mTEXT)
            f0 = predict_sentiment(s,fmodel,fTEXT)
            pre_dif_sample = abs(f0-m0)
            if pre_dif_sample>1:
                sents_dif[i] = 0.1*min(pre_dif_sample,2)
            if '<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>' in s:
                sents_dif[i] = 0.1

    loss = torch.mean(torch.mul(sents_dif,new_recon_loss))
    loss.backward()
    grad_norm = torch.nn.utils.clip_grad_norm(model.vae_params, 5)
    trainer.step()
    trainer.zero_grad()

    if iteration%2==0:
        print(iteration)
        _,sample_sent = model.forward_RL(temperature)
        sample_sent = sample_sent.split('<pad>')[0]
        print('Sample: "{}"'.format(sample_sent))
        if sample_sent:
            m0 = predict_sentiment(sample_sent,mmodel,mTEXT)
            f0 = predict_sentiment(sample_sent,fmodel,fTEXT)
            pre_dif_sample = abs(f0-m0)
            print("RL mmodel sample prediction: ",m0)
            print("RL fmodel sample prediction: ",f0)
            print("RL sample abs dif: ",pre_dif_sample)
            print("\n")

In [32]:
def train_nopad(temperature,iteration,batch_size,model):
    try:
        new_recon_loss,sample_sents = model.forward_RL_batch(temperature,batch_size)
    except:
        new_recon_loss,sample_sents = model.forward_RL_batch(temperature,batch_size)

    if temperature>1:
        temperature=temperature-0.0001

    sents_dif=torch.zeros(batch_size)
    for i,s in enumerate(sample_sents):
        s = s.split('<pad>')[0]
        if s:
            m0 = predict_sentiment(s,mmodel,mTEXT)
            f0 = predict_sentiment(s,fmodel,fTEXT)
            pre_dif_sample = abs(f0-m0)
            if '<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>' in s:
                sents_dif[i] = 0.1

        loss = 0.5*torch.mean(torch.mul(sents_dif,new_recon_loss))
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm(model.vae_params, 5)
        trainer.step()
        trainer.zero_grad()
        break

    if iteration%1==0:
        print(iteration)
        _,sample_sent = model.forward_RL(temperature)
        sample_sent = sample_sent.split('<pad>')[0]
        print('Sample: "{}"'.format(sample_sent))
        if sample_sent:
            m0 = predict_sentiment(sample_sent,mmodel,mTEXT)
            f0 = predict_sentiment(sample_sent,fmodel,fTEXT)
            pre_dif_sample = abs(f0-m0)
            print("RL mmodel sample prediction: ",m0)
            print("RL fmodel sample prediction: ",f0)
            print("RL sample abs dif: ",pre_dif_sample)
            print("\n")

In [33]:
def train_VAE(iteration,model):
    global kld_weight
    batch = next(iter(train_iter))
    inputs = batch.Text
    labels = batch.Label

    recon_loss, kl_loss = model.forward_VAE(inputs)
    loss = (recon_loss + kld_weight * kl_loss)

    # Anneal kl_weight
    if it > kld_start_inc and kld_weight < kld_max:
        kld_weight += kld_inc

    loss.backward()
    grad_norm = torch.nn.utils.clip_grad_norm(model.vae_params, 5)
    trainer.step()
    trainer.zero_grad()

    #if it % log_interival == 0:
    if iteration%10==0:
        
        z = model.sample_z_prior(1)
        sample_idxs = model.sample_sentence_VAE(z)
        sample_sent = ' '.join([TEXT.vocab.itos[i] for i in sample_idxs])
        sample_sent = sample_sent.split('<pad>')[0]
        if sample_sent:
            m0 = predict_sentiment(sample_sent,mmodel,mTEXT)
            f0 = predict_sentiment(sample_sent,fmodel,fTEXT)
            pre_dif_sample = abs(f0-m0)
        
        print('Iter-{}; VAE_Loss: {:.4f}; Recon: {:.4f}; KL: {:.4f}; Grad_norm: {:.4f};'
              .format(it, loss.data[0], recon_loss.data[0], kl_loss.data[0], grad_norm))
        print('Sample: "{}"'.format(sample_sent))
        print("VAE glove300 sample prediction: ",m0)
        print("VAE fasttext300 sample prediction: ",f0)
        print("VAE sample abs dif: ",abs(m0-f0))
        print("\n")
        

    # Anneal learning rate
    new_lr = lr * (0.5 ** (iteration // lr_decay_every))
    for param_group in trainer.param_groups:
        param_group['lr'] = new_lr

In [34]:
import spacy
nlp = spacy.load('en')

record_VAE_text=[]
record_VAE_score=[]

VAEmodel.eval()
for i in range(2000):
    z = VAEmodel.sample_z_prior(1)
    sample_idxs = VAEmodel.sample_sentence_VAE(z)
    sample_sent = ' '.join([TEXT.vocab.itos[i] for i in sample_idxs])
    sample_sent = sample_sent.split('<pad>')[0]
    if sample_sent:
        m0 = predict_sentiment(sample_sent,mmodel,mTEXT)
        f0 = predict_sentiment(sample_sent,fmodel,fTEXT)
        pre_dif_sample = abs(f0-m0)
    record_VAE_text.append(sample_sent)
    record_VAE_score.append(pre_dif_sample)

KeyboardInterrupt: 

In [ ]:
record_VAE_score = np.array(record_VAE_score)

In [ ]:
np.mean(record_VAE_score)

In [22]:
np.mean(record_VAE_score)

0.3266841918230057

In [78]:
with open('SSTmodel/allglove300_allfasttext300_VAE_sample_sents.txt','w') as f:
    for i in range(2000):
        f.write(str(record_VAE_score[i])+'\t'+record_VAE_text[i]+'\n')

In [35]:
for it in range(1,301):
    train_VAE(iteration=it,model=VAEmodel)
    #train_nopad(temperature=1,iteration=it,model=VAEmodel,batch_size=32)

/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  from ipykernel import kernelapp as app
/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iter-10; VAE_Loss: 0.8351; Recon: 0.8351; KL: 0.0001; Grad_norm: 0.0000;
Sample: "frenzy "
VAE glove300 sample prediction:  1.2519245147705078
VAE fasttext300 sample prediction:  1.6818797588348389
VAE sample abs dif:  0.42995524406433105


Iter-20; VAE_Loss: 0.8300; Recon: 0.8300; KL: 0.0002; Grad_norm: 0.0000;
Sample: "recent hollywood trip "
VAE glove300 sample prediction:  2.39509916305542
VAE fasttext300 sample prediction:  2.1294867992401123
VAE sample abs dif:  0.2656123638153076


Iter-30; VAE_Loss: 0.5689; Recon: 0.5689; KL: 0.0001; Grad_norm: 0.0000;
Sample: "filling missing "
VAE glove300 sample prediction:  2.198195457458496
VAE fasttext300 sample prediction:  1.731052041053772
VAE sample abs dif:  0.4671434164047241


Iter-40; VAE_Loss: 0.9618; Recon: 0.9618; KL: 0.0001; Grad_norm: 0.0000;
Sample: "pathos of engaging enough to be emphasized enough . "
VAE glove300 sample prediction:  3.5747549533843994
VAE fasttext300 sample prediction:  3.3366899490356445
VAE sample abs d

In [37]:
for it in range(1,1001):
    train_RL_batch(temperature=1,iteration=it,batch_size=64,model=VAEmodel)
    #train_VAE(iteration=it)

/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


2
Sample: "one of its stars "
RL mmodel sample prediction:  1.6448923349380493
RL fmodel sample prediction:  2.465059757232666
RL sample abs dif:  0.8201674222946167


4
Sample: "multi - bound urban vibe "
RL mmodel sample prediction:  3.59224009513855
RL fmodel sample prediction:  3.4962215423583984
RL sample abs dif:  0.09601855278015137


6
Sample: "last 100 years "
RL mmodel sample prediction:  2.5685319900512695
RL fmodel sample prediction:  2.2232258319854736
RL sample abs dif:  0.3453061580657959


8
Sample: "Musketeer "
RL mmodel sample prediction:  2.6905064582824707
RL fmodel sample prediction:  3.05654239654541
RL sample abs dif:  0.36603593826293945


10
Sample: "referencing of hip - hop arcana can alienate even the savviest audiences "
RL mmodel sample prediction:  2.336315631866455
RL fmodel sample prediction:  2.025498151779175
RL sample abs dif:  0.3108174800872803


12
Sample: "world 's one - star"
RL mmodel sample prediction:  2.757080316543579
RL fmodel sample predic

KeyboardInterrupt: 

In [40]:
torch.save(VAEmodel.state_dict(),'SSTmodel/{}.bin'.format('allglove300_allfasttext300_VAE_RL'))

In [74]:
fTEXT.vocab.freqs['utterly']

81

In [75]:
mTEXT.vocab.freqs['utterly']

75

In [224]:
torch.save(model.state_dict(), 'models/{}.bin'.format('pre_yelp128_100_sph_RL5'))  

In [160]:
model.load_state_dict(torch.load('models/{}.bin'.format('pre_yelp128_100_sph_RL3')))  

In [226]:
record_RL=[]
temperature = 1
for i in range(40000):
    model.eval()
    _,sample_sent = model.forward_RL(temperature)
    print('Sample: "{}"'.format(sample_sent))
    if sample_sent:
        m0 = predict_sentiment(sample_sent,mrnn,mTEXT)
        f0 = predict_sentiment(sample_sent,frnn,fTEXT)
        pre_dif_sample = abs(f0-m0)
        print("RL mrnn sample prediction: ",m0)
        print("RL frnn sample prediction: ",f0)
        print("RL sample abs dif: ",pre_dif_sample)
        print("\n")
        record_RL.append([i,pre_dif_sample,m0,f0,sample_sent])

Sample: "the food is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.618388295173645
RL frnn sample prediction:  0.8968349695205688
RL sample abs dif:  0.7215533256530762


Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.8833258152008057
RL sample abs dif:  0.4306614398956299


Sample: "yum . . i ' m from a long time . downside . worth the wait . good"
RL mrnn sample prediction:  4.180756568908691
RL frnn sample prediction:  4.041472434997559
RL sample abs dif:  0.1392841339111328


Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.8833258152008057
RL sample abs dif:  0.4306614398956299


Sample: "great indian food . taste is now closed . sign says long all . has gone to be better ."
RL mrnn sample prediction:  2.771960735321045
RL frnn sample predic

RL mrnn sample prediction:  4.844911575317383
RL frnn sample prediction:  4.531846523284912
RL sample abs dif:  0.3130650520324707


Sample: "excelllent ! ! they serve a big portion so be prepared ! ! ! !"
RL mrnn sample prediction:  4.179224967956543
RL frnn sample prediction:  4.581435203552246
RL sample abs dif:  0.4022102355957031


Sample: "love the pho ! !"
RL mrnn sample prediction:  4.576507568359375
RL frnn sample prediction:  4.660069465637207
RL sample abs dif:  0.08356189727783203


Sample: "some of the best drinks so i ' ve had in a long time . downside and the staff . staff early ."
RL mrnn sample prediction:  3.8921427726745605
RL frnn sample prediction:  3.4214558601379395
RL sample abs dif:  0.4706869125366211


Sample: "good food . lousy service . service . service . service . . this place is now closed ."
RL mrnn sample prediction:  1.721073865890503
RL frnn sample prediction:  2.9168262481689453
RL sample abs dif:  1.1957523822784424


Sample: "the food and service 

Sample: "order the food , excellent beer . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"
RL mrnn sample prediction:  4.766453742980957
RL frnn sample prediction:  4.35155725479126
RL sample abs dif:  0.41489648818969727


Sample: "bought the office and there office supplies and a special $ $ while hour ."
RL mrnn sample prediction:  3.5478339195251465
RL frnn sample prediction:  3.6492676734924316
RL sample abs dif:  0.10143375396728516


Sample: "another clean casino , around . i love to rock ! ! keep going with the busine"
RL mrnn sample prediction:  2.7474582195281982
RL frnn sample prediction:  4.824227333068848
RL sample abs dif:  2.0767691135406494


Sample: "parking is wrong but there is now closed . sign up for the players club"
RL mrnn sample prediction:  1.5272047519683838
RL frnn sample prediction:  3.2758731842041016
RL sam

Sample: "my new favorite place for sure ! !"
RL mrnn sample prediction:  4.946920871734619
RL frnn sample prediction:  4.863311767578125
RL sample abs dif:  0.08360910415649414


Sample: "love this place . karaoke is rooms and the staff is now closed ."
RL mrnn sample prediction:  4.870394706726074
RL frnn sample prediction:  4.675494194030762
RL sample abs dif:  0.1949005126953125


Sample: "awesome food . lousy service . service . service . karaoke system and no place ."
RL mrnn sample prediction:  1.947000503540039
RL frnn sample prediction:  2.579343795776367
RL sample abs dif:  0.6323432922363281


Sample: "food is now closed . wontons are good wonton soup are the soup was wow . ."
RL mrnn sample prediction:  4.57750129699707
RL frnn sample prediction:  4.245246887207031
RL sample abs dif:  0.33225440979003906


Sample: "they have good value , pizza . staff is now closed . sign says , best * dutch crunch ! ! *"
RL mrnn sample prediction:  2.2933475971221924
RL frnn sample predicti

Sample: "awesome and a buffet . staff is now closed . sign up for this place is now closed . sign on . says solid !"
RL mrnn sample prediction:  2.724466562271118
RL frnn sample prediction:  4.3212480545043945
RL sample abs dif:  1.5967814922332764


Sample: "great food . lousy service . service . service . . service is now closed ."
RL mrnn sample prediction:  2.0290021896362305
RL frnn sample prediction:  3.072662353515625
RL sample abs dif:  1.0436601638793945


Sample: "the food is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.618388295173645
RL frnn sample prediction:  0.8968349695205688
RL sample abs dif:  0.7215533256530762


Sample: "their food is now closed . sign says says reopen bad there in vegas ."
RL mrnn sample prediction:  2.9546380043029785
RL frnn sample prediction:  0.931450605392456
RL sample abs dif:  2.0231873989105225


Sample: "terrible . service . we were been to . just the menu and we will be coming back ."
RL mrnn sample pr

...

Sample: "this is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.2427709102630615
RL frnn sample prediction:  0.8241592049598694
RL sample abs dif:  0.41861170530319214


Sample: "this place is now closed . sign says says reopen in the future . better ."
RL mrnn sample prediction:  1.4955366849899292
RL frnn sample prediction:  0.8548766374588013
RL sample abs dif:  0.6406600475311279


Sample: "excellent taste , best so much fun . live music keeps it in the area . it is now closed ."
RL mrnn sample prediction:  4.679081916809082
RL frnn sample prediction:  4.695810317993164
RL sample abs dif:  0.01672840118408203


Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.8833258152008057
RL sample abs dif:  0.4306614398956299


Sample: "the food is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.618388295173645
RL frnn sampl

Sample: "home away from home . the least they ' ve been to . far from my house , fast service"
RL mrnn sample prediction:  3.893831968307495
RL frnn sample prediction:  3.527393102645874
RL sample abs dif:  0.3664388656616211


Sample: "amazing ! that 's all i need to try them without the crispy crust ."
RL mrnn sample prediction:  4.82149600982666
RL frnn sample prediction:  4.749997138977051
RL sample abs dif:  0.07149887084960938


Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.8833258152008057
RL sample abs dif:  0.4306614398956299


Sample: "all you can eat sushi . it 's a standard chain mexican food . yuuuum"
RL mrnn sample prediction:  3.4523963928222656
RL frnn sample prediction:  2.942009449005127
RL sample abs dif:  0.5103869438171387


Sample: "good food . lousy service . service . service . think could be cleaner ."
RL mrnn sample prediction:  2.173971652984619
RL frnn s

Sample: "the bacon wrapped prawns were great . . it was off a favorite . he was n't . ."
RL mrnn sample prediction:  3.7251274585723877
RL frnn sample prediction:  3.6186134815216064
RL sample abs dif:  0.10651397705078125


Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.8833258152008057
RL sample abs dif:  0.4306614398956299


Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.8833258152008057
RL sample abs dif:  0.4306614398956299


Sample: "now reopen and it . come here in check it out ."
RL mrnn sample prediction:  3.5888330936431885
RL frnn sample prediction:  4.236724853515625
RL sample abs dif:  0.6478917598724365


Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.88332581520

Sample: "this place is now closed . sign says says reopen in the future ."
RL mrnn sample prediction:  1.3139872550964355
RL frnn sample prediction:  0.8833258152008057
RL sample abs dif:  0.4306614398956299


Sample: "great place ! ! ! ! ! ! ! ! ! ! ! ! ! awesome place for coffee and friday . ."
RL mrnn sample prediction:  4.817314147949219
RL frnn sample prediction:  4.797049522399902
RL sample abs dif:  0.020264625549316406


Sample: "good food and lousy . service . service . horrible service . kitchen what 's a bit ."
RL mrnn sample prediction:  1.7418044805526733
RL frnn sample prediction:  2.1336135864257812
RL sample abs dif:  0.3918091058731079


Sample: "this place is now closed . sign says says reopen ok effective no . no no hates his stay here"
RL mrnn sample prediction:  1.0878627300262451
RL frnn sample prediction:  0.9897024631500244
RL sample abs dif:  0.0981602668762207


Sample: "great food . lousy service . service . . service . horrible service . might as well ."
RL 

RuntimeError: expected a non-empty list of Tensors

In [225]:
with open('pre_yelp128_100_sph_RL5__log','a') as f:
    for x in record_RL:
        f.write(str(x[0])+'\t'+str(x[1])+'\t'+str(x[2])+'\t'+str(x[3])+'\n')
        f.write(x[4]+'\n')

In [144]:
len(record_RL)

11792

In [30]:
record_RL = []
record_VAE = []

In [11]:
model.load_state_dict(torch.load('models/{}.bin'.format('pre_yelp128_all_sent_trainVAE_sph_less100')))  

In [28]:
model.load_state_dict(torch.load('models/{}.bin'.format('pre_yelp128_100_sph_RL')))  

In [ ]:
## pre_dif for sentences in all_sent_trainVAE_less100.tsv

In [7]:
with open('../sent/ori_gender_data/female_sent_obfandclass_less100.tsv','r') as f:
    f100 = f.readlines()
with open('../sent/ori_gender_data/male_sent_obftrain_less100.tsv','r') as f:
    m100 = f.readlines()

In [8]:
len(f100)

9707

In [9]:
len(m100)

9610

In [10]:
fpref = []
fprem = []
mpref = []
mprem = []
ftext = []
flabel = []
mtext = []
mlabel = []

In [11]:
for line in f100:
    ftext.append(line.split('\t')[0])
    flabel.append(float(line.split('\t')[1].strip('\n')))
for line in m100:
    mtext.append(line.split('\t')[0])
    mlabel.append(float(line.split('\t')[1].strip('\n')))

In [22]:
for line in ftext:
    a = predict_sentiment(line,frnn,fTEXT)
    b = predict_sentiment(line,mrnn,mTEXT)
    fpref.append(a)
    mpref.append(b)

In [23]:
for line in mtext:
    a = predict_sentiment(line,frnn,fTEXT)
    b = predict_sentiment(line,mrnn,mTEXT)
    fprem.append(a)
    mprem.append(b)

In [12]:
import numpy as np
import pandas as pd

In [13]:
with open('fpref_female_sent_obfandclass_less100tsv','r') as f:
    fpref = f.readlines()
fpref = [float(x.strip('\n')) for x in fpref]

with open('mpref_female_sent_obfandclass_less100tsv','r') as f:
    mpref = f.readlines()
mpref = [float(x.strip('\n')) for x in mpref] 

with open('fprem_male_sent_obftrain_less100tsv','r') as f:
    fprem = f.readlines()
fprem = [float(x.strip('\n')) for x in fprem]    

with open('mprem_male_sent_obftrain_less100tsv','r') as f:
    mprem = f.readlines()
mprem = [float(x.strip('\n')) for x in mprem]

In [14]:
flabel = [float(x.split('\t')[1].strip('\n')) for x in f100]
mlabel = [float(x.split('\t')[1].strip('\n')) for x in m100]
ftext = [x.split('\t')[0] for x in f100]
mtext = [x.split('\t')[0] for x in m100]

In [15]:
fpref = np.array(fpref)
fprem = np.array(fprem)
mpref = np.array(mpref)
mprem = np.array(mprem)
flabel = np.array(flabel)
mlabel = np.array(mlabel)

In [16]:
fdf = pd.DataFrame({"flabel":flabel,
                    "fpref":fpref,
                    'mpref':mpref,
                    "f-m":abs(fpref-mpref),
                    "fdiff":abs(fpref-flabel),
                    "mdiff":abs(mpref-flabel),
                    "ftext":ftext})
mdf = pd.DataFrame({"mlabel":mlabel,
                    "fprem":fprem,
                    'mprem':mprem,pejorative
                    "f-m":abs(fprem-mprem),
                    "fdifm":abs(fprem-mlabel),
                    "mdifm":abs(mprem-mlabel),
                    "mtext":mtext})

In [17]:
fdf.sort_values(by='f-m',ascending=False).head(20)

,flabel,fpref,mpref,f-m,fdiff,mdiff,ftext
7112,1.0,5.948126,1.308667,4.639459,4.948126,0.308667,place is closed
7310,1.0,5.965495,1.466454,4.499041,4.965495,0.466454,this is closed
5786,5.0,4.734744,0.590490,4.144254,0.265256,4.409510,h
7358,1.0,5.361429,1.285375,4.076054,4.361429,0.285375,this place is closed
5735,1.0,0.892889,4.893114,4.000226,0.107111,3.893114,they finally shut down !! omgggg ! thank god ....
3110,5.0,4.853196,0.870449,3.982747,0.146804,4.129551,honored our request refunded our money due to ...
2820,5.0,4.891019,0.935576,3.955443,0.108981,4.064424,wow ! how did i survive without the convenienc...
8172,1.0,0.934425,4.760951,3.826526,0.065575,3.760951,fuck a $ 20 cover . fuck house music . fuck yo...
3888,5.0,5.082398,1.263991,3.818408,0.082398,3.736009,*** this restaurant has closed . ***
5971,1.0,0.815067,4.398721,3.583654,0.184933,3.398721,sugar magnolia now closed according to the cha...


In [31]:
orisent = ftext[7358]
orisent

'this place is closed'

In [32]:
inputs = torch.LongTensor([TEXT.vocab.stoi[w] for w in orisent.split(' ')]).unsqueeze(1).to(device)

In [33]:
inputs

tensor([[  22],
        [  18],
        [  12],
        [ 177]], device='cuda:0')

In [34]:
model.eval()
mu,logvar = model.forward_encoder(inputs)
z = model.sample_z(mu,logvar)
y = model.forward_decoder(inputs, z)
reconz = model.recon_latent(inputs, z)

In [35]:
z.shape

torch.Size([1, 128])

In [36]:
y.shape

torch.Size([4, 1, 11992])

In [37]:
reconz.shape

torch.Size([4, 1, 128])

In [109]:
record_RL[0]

[0,
 0.33321285247802734,
 4.521193504333496,
 4.854406356811523,
 'for the price for that ? , i can ever had ! it was still just unbelievable .']

In [125]:
with open('RL_sph_log3','w') as f:
    for x in record_RL:
        f.write(str(x[0])+'\t'+str(x[1])+'\t'+str(x[2])+'\t'+str(x[3])+'\n')
        f.write(x[4]+'\n')